## Bibliothèques

In [2]:
import sys
import os
import numpy as numpy
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats as ttest_ind

SyntaxError: invalid syntax (<ipython-input-2-c6c3df880022>, line 6)

## Loading Files